# Function Setup

### 2.)  RNNS - Setup, Evaluation, Ensembling, etc.

In [5]:
# General function to set up a model

def create_rnn_model(model_input,nodes=[41,41],  n_output=41, final_dense_layer = True, 
                     dense_act_fct = 'linear', act_fct_special = False,
                     optimizer_type='adam', loss_type='mse', metric_type='mae', dropout_option=True, 
                     dropout_share=[0.2,0.2], lambda_layer = True, lambda_scale =1, log_scale=False, 
                     model_compile = True, return_option = 'model', branch_name = '', input_type = '3D'): #, init_style):
    
    n_layers = len(nodes)
    
    if (n_layers > len(dropout_share)) & dropout_option:
        print('No. of dropouts does not match depth of model!' )
        return
    
    if n_layers>1:
        if input_type == '3D':
            # Input suitable for RNN Layer
            x = CuDNNLSTM(units = nodes[0],return_sequences = True, 
                      name = 'Layer_{}1'.format(branch_name))(model_input)
        elif input_type == '2D':
            # Prepare Input to be suitable for RNN Layer
            x = Repeat(n = n_output)(model_input)
            x = CuDNNLSTM(units = nodes[0],return_sequences = True, 
                      name = 'Layer_{}1'.format(branch_name))(x)
        # Optional Dropout
        if dropout_option == True: 
            if dropout_share[0]>0:
                x = Dropout(dropout_share[0], name = 'Drop_{}{}_{}'.format(branch_name,1,dropout_share[0]))(x)
        for i in range(1,n_layers-1):
            x = CuDNNLSTM(units = nodes[i], return_sequences = True, name = 'Layer_'+str(i+1))(x)
            # Optional Dropout
            if dropout_option == True:
                if dropout_share[i]>0:
                    x=Dropout(dropout_share[i], 
                              name = 'Drop_{}{}_{}'.format(branch_name,i+1,dropout_share[i]))(x)
                    
        # Insert Last LSTM Layer manually and set return_sequences = False
        x=CuDNNLSTM(units=nodes[n_layers-1],return_sequences = False, 
                    name = 'Layer_{}'.format(branch_name)+str(n_layers))(x)              
    else:
        if input_type == '3D':
            # Input suitable for RNN Layer
            x = CuDNNLSTM(units = nodes[0],return_sequences = False, 
                      name = 'Layer_{}1'.format(branch_name))(model_input)
        elif input_type == '2D':
            # Prepare Input to be suitable for RNN Layer
            x = RepeatVector(n = n_output)(model_input)
            x = CuDNNLSTM(units = nodes[0],return_sequences = False, 
                      name = 'Layer_{}1'.format(branch_name))(x)
        else:
            print('Unknown input_type!')
            pass
        
    
    if final_dense_layer:
        if dropout_option == True:
            if dropout_share[n_layers-1]>0:
                x=Dropout(dropout_share[n_layers-1], 
                          name = 'Drop_{}{}_{}'.format(branch_name,n_layers, dropout_share[n_layers-1]))(x)
        # Final Dense Layer
        if act_fct_special:
            x = Dense(n_output, name = 'Layer_{}'.format(branch_name)+str(len(nodes)+1))(x)
            x = dense_act_fct(x)
        else:
            x = Dense(n_output, name = 'Layer_{}'.format(branch_name)+str(len(nodes)+1))(x)
            if dense_act_fct != 'linear':
                x = Activation(activation = dense_act_fct, name = dense_act_fct + branch_name)(x)
                
    
    if lambda_layer:
        if log_scale:
            x = Lambda(lambda x_var: tf.exp((x_var+1)/2*np.log(1+lambda_scale))-1, 
                       name = 'Log_Scaling_Layer{}'.format(branch_name))(x)
        else:
            x = Lambda(lambda x_var: (x_var+1)/2*lambda_scale, name = 'Scaling_Layer{}'.format(branch_name))(x)    
    
    #model.summary()
    if return_option == 'model':
        # Model Configuration
        model = Model(inputs=model_input, outputs=x)
        # Compile model
        if model_compile: 
            model.compile(loss = loss_type, optimizer = optimizer_type, metrics = [metric_type] )
        return model
    else:
        return x

In [1]:
# Create multiple models of the same type
# Relevant for creating an ensemble model


def create_multiple_rnn_models(number, model_input,nodes=[41,41],  n_output=41, final_dense_layer = True, 
                               dense_act_fct = 'linear', optimizer_type='adam', loss_type='mse', 
                               metric_type='mae', dropout_option=True, dropout_share=[0.2,0.2], 
                               lambda_layer = True, lambda_scale =1, log_scale=False, model_compile = True, 
                               return_option = 'model', branch_name = ''):
    
    models = []
    for i in range(number):
        models.append(create_rnn_model(model_input = model_input, nodes=nodes, n_output=n_output, 
                                       final_dense_layer = final_dense_layer, dense_act_fct = dense_act_fct, 
                                       optimizer_type=optimizer_type, loss_type=loss_type, 
                                       metric_type=metric_type, dropout_option=dropout_option, 
                                       dropout_share=dropout_share, lambda_layer = lambda_layer, 
                                       lambda_scale =lambda_scale, log_scale=log_scale, 
                                       model_compile = model_compile, return_option = return_option, 
                                       branch_name = str(i)))
    # depending on the return_option used in create_rnn_model, models can be a Tensor output or a compiled model
    return models

In [24]:
# Input: list of models
# Output: list of their weight configurations
# Purpose: Use list of weigts to transfer them to a ensemble model

def multiple_models_get_weights(models_lst):
    w = []
    for model in models_lst:
        w.append(model.get_weights())
    
    return w

In [62]:
# Train all models in a list of models

def train_individual_ensembles( models_lst, x_train, y_train, n_batch = 100, n_epochs = 20, val_share = 0.25, 
                               es_patience = 15, 
                               path = 'C:\\Users\\mark.kiermayer\\Documents\\Python Scripts\\Master Thesis - Code\\checkpoints\\Ensembles'):
    
    n_ensembles = len(models_lst)   
    model_collection = []
    hist = []
    
    # patient early stopping
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=es_patience)
    


    for i in range(n_ensembles):
        print('Training Model {} of '.format(i+1)+str(n_ensembles))
        t = time.time()

        # save only best configuration
        mc = ModelCheckpoint(filepath = path+r'\model_{}.h5'.format(i), monitor='val_loss', mode='min', 
                         verbose=0, save_best_only=True)
        hist.append(models_lst[i].fit(x_train, y_train,batch_size = n_batch, epochs = n_epochs, 
                                 validation_split = val_share, verbose=0, callbacks= [es, mc] ).history)
        # Save history
        with open(path+'\model_{}_hist.json'.format(i), 'w') as f:
            json.dump(hist[i], f )
        
        #model_collection.append(model)
        print('END of Model {}'.format(i+1)+'. Time passed: ' + str(int((time.time()-t)*100)/100) + ' sec.')
    return models_lst, hist

In [2]:
# Create a model that predicts, if a contract is still active (output 1) or matured ( output 0 )
# The magnitude of the reserve is irrelevant

def create_model_qualitative(INPUT, n_output = 41, opt='adam',loss_type= 'binary_crossentropy', 
                             metrics_type = 'accuracy', return_option = 'model', model_type = 'standard',
                             branch_name = '', input_features = 'partial', input_type = '3D', 
                             act_fct_final = 'sigmoid', option_trainable = True):
    
    # Criterion, whether input input has appropriate features (option 'partial' -> duration, age of contract) or 
    # too many (option 'all' -> age, sum insured, duration, age of contract)
    if input_features =='partial':
        INPUT_model = INPUT
    elif input_features == 'all':
        INPUT_model = Lambda(lambda x_val: x_val[:,:,2:4])(INPUT)
    else:
        print('Unknown input_features.')
        pass
    
    # Criterion if INPUT suitable for RNN (3D) or required additional preparation (2D)
    if input_type == '2D':
        INPUT_model = RepeatVector(n = n_output)(INPUT_model)
    #else:
        #do nothing
        
    
    # 'standard' equals the prepared data (for RNN-usage), where we set a default value for time points 
    # where the contract has matured
    if model_type == 'standard':
        x = Dense(units=1,activation = 'sigmoid', name = 'Layer_Qual_{}1'.format(branch_name))(INPUT_model)
        OUTPUT = Flatten(name='Transform_{}2'.format(branch_name))(x)
    # 'plain' equals a plain, repretitive input of the scaled data, i.e. features are repeated n_output-times
    elif model_type == 'plain':
        x = Dense(units=1)(INPUT_model)
        x = Flatten()(x)
        x = Dense(units=n_output, activation = 'tanh')(x)
        x = Reshape(target_shape=(n_output,1))(x)
        x = Dense(units = 1, activation = 'sigmoid')(x)
        OUTPUT = Flatten()(x)
        
    # 'plain_extended' has the same input as 'plain', but a wider layer, i.e. more neurons.
    elif model_type == 'plain_extended':
        x = Dense(units=1)(INPUT_model)
        x = Flatten()(x)
        x = Dense(units=n_output, activation = 'tanh')(x)
        OUTPUT = Dense(units = n_output, activation = 'sigmoid')(x)
    else:
        print('Unknown model_type.')
        pass
      
    if return_option == 'model':
        model = Model(inputs = INPUT, outputs = OUTPUT)
        if model_type != 'standard':
            # Use Bias to distinguish different timepoints of policy values
            dummy = (input_features == 'all')+(input_type == '2D')
            model.layers[3+dummy].set_weights([np.eye(n_output, n_output), -np.linspace(0,1,n_output)])
            model.layers[3+dummy].trainable = False
            
        # Check if model return should be trainable
        if option_trainable == False:
            for l in model.layers:
                l.trainable = False
        model.compile(optimizer=opt,loss=loss_type,metrics= [metrics_type])
        
        
        
        return model
    elif return_option == 'output':
        return OUTPUT
    else:
        print('Unknown return_option.')
        return

In [24]:
# Create a model with integrated (non-trainable) qualitative part
# Advantage: Training of Quantative Part can be focused on timepoints where contract is still active
def create_quant_qual_model(model_input,nodes=[41],  n_output=41, weights_qualitative = None, position_qualitative = 5,
                            final_dense_layer = True, 
                             dense_act_fct = 'tanh', act_fct_special = False,
                             optimizer_type='adam', loss_type='mse', metric_type='mae', dropout_option=False, 
                             lambda_layer = True, lambda_scale =1, log_scale=True, branch_name = ''):
    
    x = RepeatVector(n_output)(model_input)
    quant = create_rnn_model(model_input=x,nodes=nodes,  n_output=n_output, final_dense_layer = final_dense_layer, 
                     dense_act_fct = dense_act_fct, act_fct_special = act_fct_special,
                     optimizer_type=optimizer_type, loss_type=loss_type, metric_type=metric_type, 
                         dropout_option=dropout_option, lambda_layer = lambda_layer, lambda_scale =lambda_scale, 
                         log_scale=log_scale, return_option = 'output', branch_name = branch_name,
                         input_type = '3D')
    # Slice input for qualitative model
    y = Lambda(lambda x_val: x_val[:,:,2:4], name = 'Slicing_'+branch_name)(x)
    # Include Qualitative Model
    y = create_model_qualitative(y, return_option = 'output')
    # Binary (non-trainable !) Transformation
    qual = Lambda(lambda x_val: tf.round(x_val), name = 'Binary_Transformation')(y)
    
    # Combine quant.&qual Output
    OUTPUT = multiply([quant, qual], name='Combine_Quantitative_Qualitative_'+branch_name)
    
    model = Model(inputs=model_input, outputs=OUTPUT)     
    
    # Import qualitative model's weights
    model.layers[position_qualitative].set_weights(weights_qualitative)
    model.layers[position_qualitative].trainable = False
    #Compile model
    model.compile(loss = loss_type, optimizer = optimizer_type, metrics = [metric_type] )
    return model

In [81]:
## Combine various single-models to an ensembles model (optionally inclusively a qualitative model)
## With load_weights, the pre-trained, single models' weights can be transferred to the ensemble model
# Options for 'model_qualitative_type' are 'standard', 'plain', 'plain_extended'

def combine_models(input_layer, n_ensembles =1, model_qualitative_option = False, model_qualitative_type = 'standard',
                   load_weights = False, weights_ensembles = None, 
                   weights_qualitative = None, scale = 1, LSTM_nodes = [41,41], output_nodes = 41,
                    final_dense_layer = True, dense_act_fct = 'linear', act_fct_special = False, return_option = 'model'):
    
    # Aim: Merge a quantitative Ensemble-Model with a Qualitative classification Model
    
    output_ensemble = []
    # Create all Single Models for Ensemble
    for i in range(n_ensembles):
        model_ens =create_rnn_model(model_input=input_layer, nodes = LSTM_nodes, n_output = output_nodes,
                                    final_dense_layer = final_dense_layer, dense_act_fct= dense_act_fct,
                                    act_fct_special=act_fct_special,
                                    dropout_option=False,dropout_share=[0.2,0.2],lambda_layer = True, 
                                    lambda_scale =scale, log_scale=True, branch_name = str(i), return_option= return_option)
        
        if return_option == 'model':
        # Load weights; Not reasonable for non-model returns
            if load_weights:
                model_ens.set_weights(weights_ensembles[i])
            # Save the single-models' outputs in a list -> will be used as input to Average-Layer
            output_ensemble.append(model_ens.outputs[0])
        else:
            # For return_option 'output' ANN-objective .outputs does not exist
            output_ensemble.append(model_ens)
    
    # Combine Ensembles by Averaging them
    output_av = Average(name = 'Ensembles_Combine')(output_ensemble)
    
    if model_qualitative_option:
        

        model_qual = create_model_qualitative(INPUT = input_layer, return_option=return_option, 
                                              input_features= 'all', input_type= '3D', option_trainable= False,
                                              branch_name=str(n_ensembles), model_type= model_qualitative_type)
        if return_option == 'model':
            # Set weights for Qualitative Model
            if load_weights:
                model_qual.set_weights(weights_qualitative)

            # For return_option 'output' ANN-objectives do not exist
            output_qual = model_qual.outputs[0]
        elif return_option == 'output':
            output_qual = model_qual
        else:
            print('Unknown return_option')
            pass

        #output_qual = Lambda(lambda x: tf.cond(x>0.5,1,0), name = 'Binary_Transformation')(output_qual)
        output_qual = Lambda(lambda x: tf.round(x), name = 'Binary_Transformation')(output_qual)
        OUTPUT = multiply([output_av, output_qual], name='Combine_Quantitative_Qualitative')
    else:
        OUTPUT = output_av

    
    if return_option == 'model':
        model = Model(inputs = [input_layer], outputs = OUTPUT)
        model.compile(optimizer='adam',loss='mse',metrics= ['mae'])
        
        return model
    else:
        return OUTPUT